In [ ]:
import requests
import numpy as np
import pandas as pd

from scipy.sparse import hstack, vstack
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
url = 'https://api.arvigo.site/v1/product-recommendation'
headers = {'X-API-Key': '4a150010-bac7-46e7-8b8b-594f47b0015c'}
response = requests.get(url, headers=headers)

In [ ]:
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
# Mendefinisikan Banyaknya Rekomendasi Berdasarkan Kemiripan Produk
top_recommendation_item = 25

In [ ]:
# Membuat DataFrame dari data
data = pd.DataFrame(data['data'])
data.head()

,id,name,description,category,brand,tags,merchants,clicked
0,1,Kacamata 1,This is the description of Product 1,Glasses,Emporio,"oval, triangle, oblong","Optik Susi, Optik tik, Optik Merah Putih, Opti...",20
1,2,Kacamata 2,This is the description of Product 2,Glasses,RayBan,"triangle, square, heart","Optik Merah Putih, Optik Sukarno, Optik Susi, ...",4
2,3,Kacamata 3,This is the description of Product 3,Glasses,Emporio,"heart, triangle, oval","Optik Susi, Optik tik, Optik Sukarno, Optik Me...",10
3,4,Kacamata 4,This is the description of Product 4,Glasses,CHANEL,"heart, circle, square","Optik Susi, Optik Sukarno, Optik tik, Optik Me...",98
4,5,Kacamata 5,This is the description of Product 5,Glasses,CHANEL,"oblong, triangle, oval","Optik tik, Optik Susi, Optik Sukarno, Optik Me...",9


In [ ]:
# Menghitung vektor TF-IDF untuk masing-masing komponen dengan bobot yang berbeda
tfidf_vectorizer = TfidfVectorizer()

# Menghitung vektor TF-IDF untuk kolom 'name', 'category', 'brand', dan 'description'
name_tfidf = tfidf_vectorizer.fit_transform(data['name'])
category_tfidf = tfidf_vectorizer.fit_transform(data['category'])
brand_tfidf = tfidf_vectorizer.fit_transform(data['brand'])
description_tfidf = tfidf_vectorizer.fit_transform(data['description'])

# Menghitung vektor TF-IDF untuk kolom 'tags'
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(','), token_pattern=r"(?u)\b\w+\b")
tags_tfidf = tfidf_vectorizer.fit_transform(data['tags'])
merchants_tfidf = tfidf_vectorizer.fit_transform(data['merchants'])

# Menggabungkan matriks TF-IDF dari setiap komponen
weighted_tfidf_matrix = hstack([
    2 * category_tfidf,
    1.5 * tags_tfidf,
    1.2 * brand_tfidf,
    1 * name_tfidf,
    0.8 * description_tfidf,
    0.5 * merchants_tfidf
])

In [ ]:
# Menghitung kemiripan antar produk menggunakan cosine similarity
similarity_matrix = cosine_similarity(weighted_tfidf_matrix, weighted_tfidf_matrix)

# Membuat dictionary untuk menyimpan rekomendasi
recommendations = {}

# Menampilkan hasil kemiripan
num_products = len(data)
for i in range(num_products):
    product_id = data.loc[i, 'id']
    similar_products = [(data.loc[j, 'id'], similarity_matrix[i, j]) for j in range(num_products) if i != j]
    similar_products = sorted(similar_products, key=lambda x: x[1], reverse=True)[:top_recommendation_item]
    recommendations[product_id] = [product for product, _ in similar_products]

# Menampilkan output berupa dictionary
print(recommendations)

{1: [3, 10, 15, 5, 11, 9, 17, 4, 6, 7, 2, 8, 16, 20, 14, 12, 18, 19, 13], 2: [12, 19, 7, 18, 15, 8, 6, 14, 4, 9, 5, 1, 3, 13, 20, 16, 10, 17, 11], 3: [1, 10, 6, 5, 4, 11, 7, 9, 2, 8, 17, 18, 20, 14, 13, 12, 15, 16, 19], 4: [6, 5, 16, 3, 7, 8, 2, 10, 13, 1, 18, 9, 11, 12, 19, 15, 17, 20, 14], 5: [4, 3, 13, 1, 11, 16, 7, 6, 9, 2, 8, 17, 10, 20, 14, 18, 12, 15, 19], 6: [7, 4, 3, 2, 9, 5, 1, 8, 10, 12, 19, 16, 18, 11, 20, 14, 13, 15, 17], 7: [6, 2, 4, 12, 19, 3, 18, 20, 9, 5, 1, 8, 16, 11, 14, 13, 15, 17, 10], 8: [9, 2, 13, 4, 14, 5, 1, 3, 6, 7, 10, 17, 18, 11, 12, 19, 15, 20, 16], 9: [8, 16, 1, 7, 6, 3, 5, 20, 14, 2, 4, 17, 15, 18, 13, 11, 10, 12, 19], 10: [3, 1, 11, 4, 6, 8, 5, 18, 17, 2, 9, 7, 13, 14, 20, 12, 15, 16, 19], 11: [3, 10, 1, 5, 18, 4, 8, 7, 6, 9, 20, 14, 13, 2, 12, 15, 16, 19, 17], 12: [2, 19, 18, 15, 7, 14, 6, 4, 8, 20, 16, 13, 1, 3, 9, 5, 11, 17, 10], 13: [5, 8, 4, 14, 16, 2, 17, 7, 6, 3, 9, 12, 19, 15, 20, 18, 11, 1, 10], 14: [15, 2, 18, 13, 17, 8, 9, 12, 19, 20, 1, 7, 6,